# Transcribing Oral Testimonies with Automatic Speech Recognition
######**by Maria Dermentzi**

The purpose of this notebook is to demonstrate how you can use AI to automatically transcribe oral testimonies. We'll be using OpenAI's [Whisper](https://openai.com/research/whisper) (Radford et al., 2022) Automatic Speech Recognition (ASR) model.

#####**Credits:**
The audio files used in this tutorial were kindly provided by the [United States Holocaust Memorial Museum (USHMM)](https://www.ushmm.org/). You can follow along this tutorial by downloading them from the subdirectory named `audio` which accompanies this tutorial.

## Using ASR to transcribe and translate oral testimonies

### Installing required software

First, we install some of the libraries that we are going to be using. To execute the following code cell, hover over it and hit the **play button** or press **Shift+Enter** (or Command/Ctrl+Enter). Wait for a few seconds and a green tick appears should appear on its upper left corner, which means that the code cell has run successfully.

**Tip:** If the code cell does not execute correctly then an error message will appear typically in red. Usually, this will be due to typos in the code, missing files that haven't been uploaded or earlier code cells that were skipped but are required by following code cells. Copy and paste the error message on the search engine of your choice (e.g. Google) to troubleshoot the error.

**Note:** From now on, you will be requested to run many code cells so it is important to remember how to do so (i.e. by hitting play or using the keyboard shortcut that works for you).

In [ ]:
!apt install libcublas11
!pip install faster-whisper
!pip install pysubs2

Next, by running the following code cell we import the [`faster-whisper`](https://github.com/guillaumekln/faster-whisper) package (Klein, G. 2023) which will allow us to run Whisper in a way that is faster and requires less memory than using OpenAI's official [`whisper`](https://github.com/openai/whisper) package.

In [ ]:
from faster_whisper import WhisperModel

### Loading the Whisper model of our choice

Then, we load the Whisper model we want to use. In this notebook, by default, the third version of the `large` model is loaded. At the time of writing (January 2024), this is the most recent version of the Whisper model currently available. Read the note below and once you feel ready, execute the following code cell.

**Note:** If you're running low on compute units or if you're using the free version of Google Colab, consider using one of the smaller models by replacing `large-v3` in the code below with the name of the model you want to use (generally, the larger the model the better it performs). Make sure the name of the model is enclosed in quotation marks. If you have already run out of compute units, then comment out the code that runs Whisper on GPU runtimes by adding a `#` symbol at the start of the line and uncomment the code that lets you run Whisper on a CPU runtime by removing the `#` symbol at the start of the line.

Here are the currently availabe models, as seen in Whisper's [GitHub repo](https://github.com/openai/whisper)

|  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
|:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
|  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~1 GB     |      ~32x      |
|  base  |    74 M    |     `base.en`      |       `base`       |     ~1 GB     |      ~16x      |
| small  |   244 M    |     `small.en`     |      `small`       |     ~2 GB     |      ~6x       |
| medium |   769 M    |    `medium.en`     |      `medium`      |     ~5 GB     |      ~2x       |
| large  |   1550 M   |        N/A         |      `large`       |    ~10 GB     |       1x       |

In [ ]:
model_size = "large-v3"

# Run on GPU with FP16 precision
asr = WhisperModel(model_size, device="cuda", compute_type="float16")

# Remove the "#" symbol below to run on GPU with INT8 precision
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")

# Remove the "#" symbol below to run on CPU with INT8 precision
# model = WhisperModel(model_size, device="cpu", compute_type="int8")

Before we start transcribing, we define a function which we'll use to obtain the results of the transcriptions in the form that we want them to be, ie as timestamped segments and as text. Don't forget to execute the following cell.

In [ ]:
def get_transcript(segments):
  result = {
      'segments':[],
      'text': ""
  }

  for segment in segments:
      print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
      result['segments'].append({'start': segment.start, 'end': segment.end, 'text': segment.text})
      result['text']+=segment.text

  return result

### Uploading files to transcribe

Next, we need to specify an audio or video file to transcribe and/or translate. This can be done by either uploading a file to this session or by adding a link to the file we want to transcribe. Most major audio and video file types are supported.

**Reminder:** You can use the same files as we use in this tutorial by downloading them from the `audio` folder accompanying this tutorial.

The following cell assumes that a file has been uploaded in the current colab environment. We can do this by clicking on the "Files" icon on the vertical toolbar on the left of colab's interface. Then, click the "Upload to session storage" icon (the icon depicting a document with an upward-pointing arrow) and select your file or files. At this stage, if you want to follow this tutorial using the excerpts of the USHMM oral testimonies found in the accompanying folder, select all of the audio files you have previously downloaded. Google Colab might give you a warning regarding the fact that your files will not be saved permanently if uploaded this way—you can ignore this warning.

Wait a few seconds until the files are uploaded. If you cannot see the files in the `Files` directory, click on the `Refresh` icon (the icon that depicts a filder with an refresh arrow).

Next, copy the path to the `thomas.mov` file (or to file that you have uploaded in case you are not using the accompanying audio files) by right-clicking onto it and clicking on `Copy Path`. Next, paste the path that you've copied in the code cell below. Make sure it is pasted between the quotation marks (it needs to be enclosed in quotation marks). Once ready, execute the following code cell that defines the path to the file.

**Tip:** Alternatively, if there is a public URL to your audio file, you can copy and paste the URL to it directly (i.e. without having to upload it to the notebook's session storage).

In [ ]:
thomas_path = ""

### Transcribing interviews in English

The first audio file that we're going to transcribe is an excerpt of the [Oral history interview with Peter Thomas](https://collections.ushmm.org/search/catalog/irn512162). In this recording, Peter Thomas, an American soldier, describes his experience during WWII. The recording was made in perfect conditions, the person speaking is a native English speaker with a very clear American accent, and it is him narrating something rather than speaking to an interviewer.

The next code cell is where we'll feed our audio file to the model. Feel free to run the following code cell. Once this process it finished, the result is going to be saved in the `thomas_result` variable.

**Note:** In case you want to transcribe your own audio file that you have uploaded to this notebook's session storage, rest assured that the fact that our variables here are named `thomas_path` and `thomas_result` does not mean that your audio file will not get transcribed. We could give any name to our variables. The important part is that you copy and paste the right path to your file. The same is true for the rest of the examples throughout this tutorial.

In [ ]:
segments, info = asr.transcribe(thomas_path)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

thomas_result = get_transcript(segments)

Unsurprisingly, the result is very good if not flawless. This proves how powerful Whisper can be when the conditions of the recording are ideal and when the speech is in clear American English or any other mainstream accent.

### Transcribing interviews in English using the `initial_prompt` parameter to guide the model towards more accurate results

Next, we will transcribe the ['Oral history interview with Mary Nahmias'](https://collections.ushmm.org/search/catalog/irn510123) which belongs to The Jeff and Toby Her Oral History Archive at the USHMM. It seems to have taken place in or (most likely) before 1995 and was later digitised. The interviewee speaks in English with a Greek accent.

Copy the path of the file `nahmias.m4a` and paste it within the quotation marks below. Once ready, hit run.

In [ ]:
nahmias_path = ""


Here, apart from the audio file parameter (`nahmias_path`), we also specify the `initial_prompt` parameter. The `initial_prompt` is optional but can guide the model in producing more accurate results. It is more useful when the speech includes words like abbreviations to help the model transcribe them in the correct form. In this case, the model tends to transcribe "hair" as "heir" or as "ribs" (!) depending on the version of Whisper we are using, and so we prompt it to guide it towards selecting the correct word. Run the following code cell.

In [ ]:
segments, info = asr.transcribe(nahmias_path, initial_prompt="They cut all our hair...")

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

nahmias_result = get_transcript(segments)

The following line will dislpay the resulting transcript as one string of text without the timestamps. Hit run.

In [ ]:
nahmias_result['text']

### Generating subtitle files

Let's say that you want to generate an srt file out of this transcript to enhance your audio/video file with subtitles. Hit run to see how you can do this:

In [ ]:
import pysubs2
# To produce srt files for other transcripts replace nahmias_result['segments']
# with the name of the other transcript

subs = pysubs2.load_from_whisper(nahmias_result['segments'])

# You can change the name of the srt file by replacing nahmias.srt with another name
subs.save('nahmias.srt')

You can find the srt file in the content directory where we you've previously uploaded the audio. If you cannot find it, press the refresh icon. By clicking on the three dots, you can download the .srt file on your computer.

### Transcribing interviews that are not in English

Whisper can automatically detect language based on the first 30 seconds of the file. If we specify `language = None`, then Whisper will perform automatic language detection. This process does not work correctly if there is no speech in the first 30 seconds. However, if you know the language spoken in your file or if there are more languages than one and you're mostly interested in a language that is not the main one spoken during the first 30 seconds, it is recommended to specify the name of your preferred language. By leaving the following cell as is (`language = "ro"`), Whisper will assume that the file is in Romanian. Ideally, the language must be entered in its ISO 639-1 form. You can find the codes on [Wikipedia](https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes).

Remember to alway enclose the name of the language in quotes, for example:

```
language = "ro"
```

For supported languages and Whisper's Word Error Rate (WER) per language see [here](https://github.com/openai/whisper/blob/main/language-breakdown.svg) (the lower the error rate, the better).

Run the following code cell to proceed.

In [ ]:
language = "ro"

#### Transcription and English translation

Next, let's try to transcribe and translate part of the [Oral history interview with Radu Frunză](https://collections.ushmm.org/search/catalog/irn517711) which is accessible online through USHMM. The interview is in Romanian and there is a lot of background noise. The path to the file has already been specified for you but you still need to run the code cell.

In [ ]:
frunza = '/content/frunza.m4a'

The following line of code will run Whisper on the specified audio file and translate it to English. The results of this run will get saved in the `frunza_result` variable. This time we'll try to transcribe it by specifying the language as `ro` (ISO code for Romanian). Notice we also specify the task that we want Whisper to perform, which in this case is `"translate"`. Next, run the following code cell.

In [ ]:
segments, info = asr.transcribe(frunza, task="translate", language=language)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

frunza_result = get_transcript(segments)

The resulting transcript has a few mistakes in it and this is a reminder that it is still necessary to verify and edit the transcripts as necessary once they are obtained.

#### Transcription in the original (non-English) language

But what if we want the transcript to be in another language? Although Whisper allows you to get a transcript in the original language of the file, it does not support translation to any language other than English at the moment. To obtain a transcript in the original language, you simply run the transcription task, like we do in the code cells below, where we transcribe part of USHMM's [Research interview with Josif Stroumsa](https://collections.ushmm.org/search/catalog/irn629223). Feel free to run all of the remaining code cells.

In [ ]:
stroumsa = "/content/stroumsa.m4a"

In [ ]:
segments, info = asr.transcribe(stroumsa)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

stroumsa_result = get_transcript(segments)

In [ ]:
stroumsa_result['text']

For readers of this tutorial who understand Greek, you might notice that there are at least three typos in the generated transcript, but overall Whisper is performing quite well with Greek audio files. Despite the overall good performance, it is still necessary to verify and edit the transcripts to ensure there are no mistakes.

### Thank you very much for making it to the end of this tutorial!

If you have any questions, email me at maria.1.dermentzi@kcl.ac.uk

Maria Dermentzi

## References
Klein, G. (2023). Faster Whisper transcription with CTranslate2 [Python]. https://github.com/guillaumekln/faster-whisper (Original work published 2023)

Radford, A., Kim, J. W., Xu, T., Brockman, G., McLeavey, C., & Sutskever, I. (2022). Robust Speech Recognition via Large-Scale Weak Supervision (arXiv:2212.04356). arXiv. https://doi.org/10.48550/arXiv.2212.04356

# How to Cite this Tutorial:

Dermentzi, M. (2024, January). Transcribing Oral Testimonies with Automatic Speech Recognition. EHRI Collection of Digital Tools Guides. https://github.com/EHRI/ehri-data-analysis-tools/tree/master/automatic-speech-recognition